In [1]:
import torch
import numpy as np
import sys

sys.path.append('../utils/human_body_prior/tools/')
from omni_tools import copy2cpu as c2c
from os import path as osp

both57m_dir = '../data/both57m/'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(comp_device)

cuda


In [2]:
sys.path.append('../utils/human_body_prior/body_model/')
from body_model import BodyModel

gender = "male"

bm_fname = '../models/smplh_models/{}/model.npz'.format(gender)
bm = BodyModel(bm_fname=bm_fname).to(comp_device)
faces = c2c(bm.f)

seq_num = 4

In [3]:
data_ranges = [range(70,76),range(493,496),range(2074,2079),range(2843,2845),range(3000,3003)]
start_offsets = [30,38,60,0,60]
end_offsets = [0,52,0,0,20]

root_orient = np.empty((0,3))
body_pose = np.empty((0,21,3))
hand_pose = np.empty((0,30,3))
trans = np.empty((0,3))
seq_len = 0

In [4]:
data_range = data_ranges[seq_num]
start_offset = start_offsets[seq_num]
end_offset = end_offsets[seq_num]

for data_number in data_range:
    both57m_npz = osp.join(both57m_dir, str(data_number), 'data.npz')
    bdata = np.load(both57m_npz)

    seq_len += bdata['seq_len']
    root_orient = np.concatenate((root_orient, bdata['root_orient']))
    body_pose = np.concatenate((body_pose, bdata['body_pose']))
    hand_pose = np.concatenate((hand_pose, bdata['hand_pose']))
    trans = np.concatenate((trans,bdata['trans']))                                   

In [5]:
final_len = seq_len - start_offset - end_offset

motion_parms = {
    'root_orient': torch.Tensor(root_orient[start_offset:start_offset+final_len,:]).to(comp_device),
    'pose_body': torch.Tensor(body_pose[start_offset:start_offset+final_len].reshape(final_len,-1)).to(comp_device),
    'pose_hand': torch.Tensor(hand_pose[start_offset:start_offset+final_len].reshape(final_len,-1)).to(comp_device),
    'trans': torch.Tensor(trans[start_offset:start_offset+final_len,:]).to(comp_device)
}

print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in motion_parms.items()])))

Body parameter vector shapes: 
root_orient: torch.Size([280, 3]) 
pose_body: torch.Size([280, 63]) 
pose_hand: torch.Size([280, 90]) 
trans: torch.Size([280, 3])


In [6]:
import trimesh
sys.path.append('../utils/body_visualizer')
from tools.vis_tools import colors
from mesh.mesh_viewer import MeshViewer
from mesh.sphere import points_to_spheres
from tools.vis_tools import show_image

imw, imh=1600, 1600
mv = MeshViewer(width=imw, height=imh, use_offscreen=True)

In [7]:
body_pose = bm(**{k:v for k,v in motion_parms.items() if k in ['pose_body', 'pose_hand', 'root_orient','trans']})

def vis_body_hand_pose(fId = 0, SHOW=True):
    body_mesh = trimesh.Trimesh(vertices=c2c(body_pose.v[fId]), faces=faces, vertex_colors=np.tile(colors['grey'], (6890, 1)))
    mv.set_static_meshes([body_mesh])
    body_image = mv.render(render_wireframe=False)
    if SHOW: show_image(body_image)

    return body_image

body_images = []
for i in range(final_len):
    body_images.append(vis_body_hand_pose(fId=i, SHOW=False))

In [8]:
import mediapy as media

media.show_video(images=body_images,fps=15,width=700)

This browser does not support the video tag.
